In [1]:
import os
import pathlib
import shutil
import json
import polars as pl
from pprint import pprint
from tqdm import tqdm

pl.Config.load_from_file("pl_config.json")
pl.set_random_seed(999)

## 다운로드

다운받을 데이터셋 정보 조회

In [2]:
!aihubshell -mode l -datasetkey 127

aihubshell version 24.01.29 v0.3
Fetching file tree structure...
The contents are encoded in UTF-8 including Korean characters. 
If the following contents are not output normally, 
Please modify the character information of the OS. 
=================공지사항=================== 

    └─029.한국어-일본어 번역 말뭉치
        ├─01.데이터
        │  ├─2.Validation
        │  │  ├─원천데이터
        │  │  │  └─ko2ja_culture_validation_csv.zip | 24 MB | 46377
        │  │  └─라벨링데이터
        │  │      └─ko2ja_culture_validation_json.zip | 26 MB | 46376
        │  └─1.Training
        │      ├─라벨링데이터
        │      │  └─ko2ja_culture_training_json.zip | 209 MB | 46374
        │      └─원천데이터
        │          └─ko2ja_culture_training_csv.zip | 191 MB | 46375
        ├─02.저작도구
        │  ├─2.저작도구파일.zip | 725 MB | 396896
        │  └─1.사용매뉴얼.zip | 26 MB | 396897
        └─AI모델 및 소스코드.zip | 409 MB | 396898


데이터 개요는 이 [페이지](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=127)에서 찾아볼 수 있다. 라벨링데이터를 다운받아 분석을 진행한다.

In [3]:
!aihubshell -mode d -datasetkey 127 -filekey 46374

aihubshell version 24.01.29 v0.3
Authentication successful.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  208M    0  208M    0     0  5396k      0 --:--:--  0:00:39 --:--:-- 6606k  0  4506k      0 --:--:--  0:00:14 --:--:-- 5708k:32 --:--:-- 6176k
Request successful with HTTP status 200.
Download successful.
x 029.한국어-일본어_번역_말뭉치/01.데이터/1.Training/라벨링데이터/ko2ja_culture_training_json.zip.part0
잠시 기다려 주세요 병합중 입니다. 
Merging ko2ja_culture_training_json.zip in ./029.한국어-일본어_번역_말뭉치/01.데이터/1.Training/라벨링데이터
병합이 완료 되었습니다. 


다운받은 데이터 디렉토리 및 결과 디렉토리 정의

In [4]:
archive_dir = [path for path in pathlib.Path(os.getcwd()).glob("029*") if path.suffix != ".ipynb"][0]
result_dir = pathlib.Path(f"{os.getcwd()}/aihub-datasets")
result_dir.mkdir(parents=True, exist_ok=True)

## 압축 해제 및 EDA

데이터 압축파일 경로 정의

In [5]:
for walk in os.walk(archive_dir):
    current_path, folders, files = walk
    for file_name in files:
        if file_name.endswith(".zip") and current_path.endswith("라벨링데이터"):
            archive_path = f"{current_path}/{file_name}"

압축 해제

In [6]:
data_dir = pathlib.Path(archive_path.strip(".zip"))
shutil.unpack_archive(
    filename=archive_path,
    extract_dir=data_dir,
    format="zip",
)

각 데이터는 json 형태로 저장되어 있다.

In [7]:
data_files = list(data_dir.glob("*.json"))
print(f"Number of files in dataset : {len(data_files):,}")

Number of files in dataset : 9


데이터 속성값들. 카테고리별로 한국어-일본어 문장 페어가 `출처`, `일본어_글자수`, `한국어_어절수`와 같은 특징들과 함께 포함되어 있는 것을 확인할 수 있다.

In [8]:
with open(data_files[0], "r") as file:
    documents = json.load(file)
pprint(documents[:3])

[{'관리번호': 'KO-JA-2020-FINA-000501',
  '길이_분류': 4,
  '분야': '금융/증시',
  '수행기관': '플리토',
  '일본어': '金融投資業界によると、転売を中断したライム資産運用の私募ファンドは「プルートーFI '
         'D-1号」「テティス2号」「プルートーTF-1号」「クレジットエンシュアド1号」であり、この内に不法行為の相当部分が発見されたファンドは貿易金融ファンドだ。',
  '일본어_글자수': 93,
  '출처': 'http://www.sedaily.com/NewsView/1YYZENADHR',
  '한국어': "금융투자업계에 따르면 환매를 중단한 라임자산운용의 사모펀드는 '플루토FI D-1호', '테티스 2호', '플루토TF- "
         "1호', '크레디트 인슈어드 1호'로 이중 불법행위 상당 부분이 발견된 펀드는 무역금융펀드다.",
  '한국어_어절수': 22},
 {'관리번호': 'KO-JA-2020-FINA-000502',
  '길이_분류': 2,
  '분야': '금융/증시',
  '수행기관': '플리토',
  '일본어': 'ところで被害者ではない方々にもホン記者が伝達しなければならない話があるのですよ。',
  '일본어_글자수': 39,
  '출처': 'https://www.nocutnews.co.kr/news/5290854',
  '한국어': '그런데 피해자가 아닌 분들에게도 홍 기자가 전달해야 할 말이 있다고요.',
  '한국어_어절수': 10},
 {'관리번호': 'KO-JA-2020-FINA-000503',
  '길이_분류': 3,
  '분야': '금융/증시',
  '수행기관': '플리토',
  '일본어': '待ち時間なしに通帳・デビットカードの新規発給、OTPカード発給、各種申告などができ、ラウンジマネージャーがデジタル機器の利用を支援する。',
  '일본어_글자수': 56,
  '출처': 'http://www.sedaily.com/NewsView/1YYX3XWKFE',
  '한국어': '대기시간 없이 통장·체크카드 신

전체 로드

In [9]:
corpus = []
for data_file in tqdm(data_files):
    with open(data_file, "r") as file:
        documents = json.load(file)
    for document in documents:
        data = {
            "korean": document["한국어"],
            "japanese": document["일본어"],
            "category": document["분야"]
        }
        corpus.append(data)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:03<00:00,  2.75it/s]


EDA를 위해 데이터프레임으로 변환

In [10]:
eda_dataset = pl.DataFrame(corpus).sample(fraction=1, shuffle=True)
print(f"Number of documents in the corpus : {eda_dataset.height:,}")

Number of documents in the corpus : 1,200,000


### 카테고리별 문서 수 분포

카테고리별 문서 수 분포는 아래와 같았다.

In [11]:
eda_dataset.group_by("category").len("num_documents")

category,num_documents
str,u32
"""금융/증시""","160,000"
"""IT/컴퓨터/모바일""","160,000"
"""사회/노동/복지""","160,000"
"""의료/보건""","80,000"
"""교육""","80,000"
"""특허/기술""","120,000"
"""K-POP(한류)/대중문화공연콘텐츠""","160,000"
"""문화재/향토/K-FOOD""","120,000"
"""자동차/부품/교통/소재""","160,000"


### 문서 토큰 수 분포 확인

한국어에서는 정확히 적용되지 않지만, 편의를 위해 어절의 수(`공백 수 + 1`)를 토큰 수라고 정의한다. 한국어 토큰이 약 1,900만개 가량 포함된 corpus임을 확인했다.

In [12]:
eda_dataset = eda_dataset.with_columns(pl.col("korean").str.replace(r"\s+", " "))
num_tokens = (
    eda_dataset
    .with_columns((pl.col("korean").str.count_matches(" ") + 1).alias("num_tokens"))
    .select("num_tokens")
)
print(f"Number of tokens in the Korean corpus : {num_tokens['num_tokens'].sum():,}")

Number of tokens in the Korean corpus : 18,784,762


문서당 토큰 수 분포

In [13]:
num_tokens.select(
    pl.col("num_tokens").min().alias("min"),
    pl.col("num_tokens").quantile(0.05).cast(pl.Int32).alias("q05"),
    pl.col("num_tokens").quantile(0.25).cast(pl.Int32).alias("Q1"),
    pl.col("num_tokens").quantile(0.50).cast(pl.Int32).alias("median"),
    pl.col("num_tokens").quantile(0.75).cast(pl.Int32).alias("Q3"),
    pl.col("num_tokens").quantile(0.95).cast(pl.Int32).alias("q95"),
    pl.col("num_tokens").max().alias("max"),
)

min,q05,Q1,median,Q3,q95,max
u32,i32,i32,i32,i32,i32,u32
3,8,11,15,19,25,35


### 마스킹 토큰 존재 여부 확인

다른 AI 허브 데이터같이 비식별화가 필요한 텍스트에 마스크가 적용되어 있는지 확인했다. 없다.

In [14]:
(
    eda_dataset
    .with_columns(pl.col("korean").str.extract(r"(\([가-힣]+\))").alias("mask_pattern"))
    .filter(pl.col("mask_pattern").is_not_null())
    .group_by("mask_pattern")
    .len("pattern_count")
    .sort(by="pattern_count", descending=True)
    .head(10)
)

mask_pattern,pattern_count
str,u32
"""(수)""",665
"""(금)""",664
"""(월)""",553
"""(목)""",486
"""(화)""",411
"""(주)""",107
"""(앱)""",102
"""(토)""",85
"""(여자)""",71


## 데이터 저장

EDA를 마친 corpus를 앞서 정의한 `result_dir`에 같은 다운받은 데이터셋과 동일한 이름으로 저장

In [15]:
result_path = result_dir.joinpath(f"{archive_dir.name}.parquet")
eda_dataset.write_parquet(result_path)

로컬 디렉토리 정리

In [16]:
os.system(f"rm -rf {archive_dir}")

0

저장된 데이터셋에는 한국어 문장(`korean`), 이에 대응하는 일본어 문장(`japanese`)과 함께 해당 문장의 카테고리(`category`)가 저장되어 있다.

In [17]:
eda_dataset.head(10)

korean,japanese,category
str,str,str
"""지원 내용에 비해 채무자가 감수해야 할 불이익이 크다.""","""支援内容に比べて債務者が甘受しなければならない不利益が大きい。""","""금융/증시"""
"""""상품을 사러 꼭 동대문에 가야 하나요?"" ""고객 불만 전화를 누가 대신 받아줬으면 좋겠어요."" 온라인에서 장사하는 사업자들에…","""「商品を買いに必ず東大門に行かなければならないのですか?」「顧客の不満電話を誰か代わりに取ってほしいです。」オンラインで商売をする事業者から…","""IT/컴퓨터/모바일"""
"""시·도교육청들이 지난 9일 기준으로 중3과 고3 숫자를 보고해 교육부가 취합한 숫자다.""","""市・道教育庁が9日を基準に中3と高3の数字を報告し、教育部が集合した数字だ。""","""사회/노동/복지"""
"""우리은행에 대한 모럴해저드 비판은 비밀번호 도용사건이 발생한 지 약 1년 후 벌어진 '해외 금리연계 파생결합펀드'(DLF) 사…","""ウリィ銀行に対するモラルハザードの批判は暗証番号盗用事件が発生してから約1年後に起きた「海外金利連係派生結合ファンド」(DLF)事態の時も提…","""금융/증시"""
"""이어 이 재판 과정에서 위증을 교사한 혐의로 추가 기소돼 징역 1년의 실형까지 확정됐다.""","""続いてこの裁判過程で偽証を教唆した疑いで追加起訴され、懲役1年の実刑まで確定された。""","""사회/노동/복지"""
"""본 발명은 압전 트랜스를 성형 제조시 별도의 모서리부 면취 가공등을 삭제하여 압전 트랜스 제조공정을 단순화할 수 있도록한 압전…","""本発明は、圧電トランスを成形製造する時、別途の角部面取り加工などを削除して圧電トランス製造工程を単純化できるようにした圧電トランスの製造方法…","""특허/기술"""
"""공단은 중앙노동위의 이 같은 단서를 근거로 복직 10개월 만에 인사위원회를 열고 중징계 처분을 내렸다.""","""公団は、中央労働委のこのような手がかりを根拠に、復職10ヵ月後に人事委員会を開き、重い懲戒処分を下した。""","""사회/노동/복지"""
"""한국 사회는 경제 성장으로 인하여 새로운 것과 고가품에 대한 수요가 급증하였고, 옷을 패션으로 인식하며 끊임없이 새로운 것을 …","""韓国社会は経済成長によって新しいものと高価品に対する需要が急増し、服をファッションとして認識し、絶えず新しいことを探してきた。""","""문화재/향토/K-FOOD"""
"""또한, 다이어프램의 상부로 설치되는 디스크판 역시 저면 외측단부에 접촉안전수단이 형성되므로, 다이어프램의 상, 하 작동시 접촉…","""また、ダイアフラムの上部に設置されるディスク板も底面外側端部に接触安全手段が形成されるので、ダイアフラムの上・下作動時の接触によるダイアフラ…","""특허/기술"""
